In [16]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen

In [22]:
# --- CONFIG ---
os.environ["DB_USER"] = "postgres"
os.environ["DB_PASS"] = "AzuleneLabs_2026"
os.environ["DB_HOST"] = "azulene-1.cizeysmsgxmm.us-east-1.rds.amazonaws.com"
os.environ["DB_NAME"] = "postgres"
os.environ["DB_PORT"] = "5432"

print(os.getenv("DB_USER"))  # confirm


DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_PORT = os.getenv("DB_PORT", 5432)

print("🔧 Configuration: ")
print(f"DB_USER: {DB_USER}")
print(f"DB_HOST: {DB_HOST}")
print(f"DB_NAME: {DB_NAME}")
print(f"DB_PORT: {DB_PORT}")

BATCH_SIZE = 1000

# --- Connect to PostgreSQL ---
conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(conn_str)


# --- RDKit-based calculations ---
def compute_rdkit_features(smiles):
    """Compute molecular weight, logP, etc. using RDKit."""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            return None
        return {
            "mol_weight": Descriptors.MolWt(mol),
            "logp_rdkit": Crippen.MolLogP(mol)
        }
    except Exception:
        return None


# --- PubChem fallback API ---
def fetch_pubchem_logp(smiles):
    """Fetch experimental logP from PubChem."""
    try:
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{smiles}/property/LogP,IsomericSMILES/JSON"
        resp = requests.get(url, timeout=5)
        if resp.status_code == 200:
            props = resp.json().get("PropertyTable", {}).get("Properties", [{}])[0]
            return props.get("LogP")
    except Exception:
        return None


# --- Load data ---
def fetch_data(limit=None):
    """Fetch data from PostgreSQL table."""
    query = "SELECT * FROM drug_properties"
    if limit:
        query += f" LIMIT {limit}"
    return pd.read_sql(query, engine)


# --- Enrichment Pipeline ---
def enrich_dataframe(df):
    enriched = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Enriching molecules"):
        smiles = row.get("smiles")
        data_origin = {}

        if not smiles:
            enriched.append(row)
            continue

        # --- Compute with RDKit ---
        rdkit_features = compute_rdkit_features(smiles)
        if rdkit_features:
            if pd.isna(row.get("logp")) and rdkit_features["logp_rdkit"] is not None:
                row["logp"] = rdkit_features["logp_rdkit"]
                data_origin["logp"] = "rdkit"
            if pd.isna(row.get("binding_free_energy")):
                # (placeholder example)
                row["binding_free_energy"] = -0.1 * rdkit_features["logp_rdkit"]
                data_origin["binding_free_energy"] = "estimated_rdkit"

        # --- PubChem fallback ---
        if pd.isna(row.get("logp")):
            logp_pubchem = fetch_pubchem_logp(smiles)
            if logp_pubchem is not None:
                row["logp"] = logp_pubchem
                data_origin["logp"] = "pubchem"

        # --- Simple pKa estimation (toy model) ---
        if pd.isna(row.get("pka")) and rdkit_features:
            row["pka"] = 7.0 - 0.2 * rdkit_features["logp_rdkit"]
            data_origin["pka"] = "estimated_rdkit"

        # --- Solubility fallback (basic logS estimation) ---
        if pd.isna(row.get("solubility")) and rdkit_features:
            logp = rdkit_features["logp_rdkit"]
            molwt = rdkit_features["mol_weight"]
            row["solubility"] = -0.01 * molwt - 0.5 * logp
            data_origin["solubility"] = "estimated_rdkit"

        # Track origin of each field
        row["metadata"] = json.dumps({"data_origin": data_origin})
        enriched.append(row)

    return pd.DataFrame(enriched)


# --- Main workflow ---
if __name__ == "__main__":
    print("🚀 Loading data from database...")
    df = fetch_data(limit=5000)  # adjust for testing
    print(f"Loaded {len(df)} molecules.")

    print("🔬 Enriching data...")
    enriched_df = enrich_dataframe(df)

    print(enriched_df.head())

    '''print("💾 Saving to database...")
    save_to_postgres(enriched_df)

    print("✅ Enrichment complete!")'''


postgres
🔧 Configuration: 
DB_USER: postgres
DB_HOST: azulene-1.cizeysmsgxmm.us-east-1.rds.amazonaws.com
DB_NAME: postgres
DB_PORT: 5432
🚀 Loading data from database...
Loaded 5000 molecules.
🔬 Enriching data...


Enriching molecules: 100%|██████████| 5000/5000 [00:05<00:00, 926.05it/s] 


      chembl_id                                            smiles  \
0    CHEMBL6329      Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl   
1    CHEMBL6328   Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1   
2  CHEMBL265667  Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1   
3    CHEMBL6362      Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1   
4  CHEMBL267864    Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1   

   binding_free_energy  solubility     logp permeability       pka  \
0            -0.211362    -4.47435  2.11362         None  6.577276   
1            -0.133190    -3.98914  1.33190         None  6.733620   
2            -0.227274    -4.71434  2.27274         None  6.545452   
3            -0.146022    -3.80320  1.46022         None  6.707956   
4            -0.211362    -4.47435  2.11362         None  6.577276   

  molecular_weight   hba   hbd   psa   rtb qed_weighted  source  \
0             None  None  None  None  None         None  ChEMBL   
1             None  None  None

In [23]:
enriched_df.head()

,chembl_id,smiles,binding_free_energy,solubility,logp,permeability,pka,molecular_weight,hba,hbd,psa,rtb,qed_weighted,source,metadata
0,CHEMBL6329,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl,-0.211362,-4.47435,2.11362,None,6.577276,None,None,None,None,None,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre..."
1,CHEMBL6328,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1,-0.133190,-3.98914,1.33190,None,6.733620,None,None,None,None,None,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre..."
2,CHEMBL265667,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1,-0.227274,-4.71434,2.27274,None,6.545452,None,None,None,None,None,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre..."
3,CHEMBL6362,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1,-0.146022,-3.80320,1.46022,None,6.707956,None,None,None,None,None,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre..."
4,CHEMBL267864,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1,-0.211362,-4.47435,2.11362,None,6.577276,None,None,None,None,None,None,ChEMBL,"{""data_origin"": {""logp"": ""rdkit"", ""binding_fre..."


In [24]:
enriched_df.keys()

Index(['chembl_id', 'smiles', 'binding_free_energy', 'solubility', 'logp',
       'permeability', 'pka', 'molecular_weight', 'hba', 'hbd', 'psa', 'rtb',
       'qed_weighted', 'source', 'metadata'],
      dtype='object')

In [34]:
def save_to_postgres(df):

    ## Make sure all metadata rows are strings
    df["metadata"] = df["metadata"].apply(
        lambda x: json.dumps(x) if isinstance(x, dict) else x
    )

    df.to_sql("drug_properties_enriched", engine, if_exists="replace", index=False)
    print(f"✅ Saved {len(df)} enriched rows to drug_properties_enriched table.")

In [35]:
save_to_postgres(enriched_df)

✅ Saved 5000 enriched rows to drug_properties_enriched table.


In [28]:
type(enriched_df['metadata'][0])

str

In [29]:
enriched_df["metadata"].apply(type).value_counts()

metadata
<class 'str'>     4961
<class 'dict'>      39
Name: count, dtype: int64

In [33]:
for key in enriched_df.keys():
    print(enriched_df[key].apply(type).value_counts())
    print()

chembl_id
<class 'str'>    5000
Name: count, dtype: int64

smiles
<class 'str'>         4961
<class 'NoneType'>      39
Name: count, dtype: int64

binding_free_energy
<class 'float'>    5000
Name: count, dtype: int64

solubility
<class 'float'>    5000
Name: count, dtype: int64

logp
<class 'float'>    5000
Name: count, dtype: int64

permeability
<class 'NoneType'>    5000
Name: count, dtype: int64

pka
<class 'float'>    5000
Name: count, dtype: int64

molecular_weight
<class 'NoneType'>    5000
Name: count, dtype: int64

hba
<class 'NoneType'>    5000
Name: count, dtype: int64

hbd
<class 'NoneType'>    5000
Name: count, dtype: int64

psa
<class 'NoneType'>    5000
Name: count, dtype: int64

rtb
<class 'NoneType'>    5000
Name: count, dtype: int64

qed_weighted
<class 'NoneType'>    5000
Name: count, dtype: int64

source
<class 'str'>    5000
Name: count, dtype: int64

metadata
<class 'str'>     4961
<class 'dict'>      39
Name: count, dtype: int64

